# Step 1 - Importing our data

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import locale

cities = ['2800', '2820', '2830', '2840', '2850', '2900', '2920', '2930', '2942', '2950', '3000', '3460']
dataframes = []

for city in cities:
    filename = f'./data/house_data/house_data_{city}.csv'
    df = pd.read_csv(filename)
    df["City"] = int(city)
    dataframes.append(df)
data = pd.concat(dataframes, ignore_index=True)

data.head(5)

,Address,X,Y,Price,Type,Size,Squaremeter price,Energy class,Url,City
0,"Nørgaardsvej 27, 2. th",55.770647,12.508193,1595000,Ejerlejlighed,44,36250,E,https://www.dingeo.dk/adresse/2800-kongens%20l...,2800
1,Gammelmosevej 218,55.756103,12.480264,4495000,Rækkehus,113,39778,C,https://www.dingeo.dk/adresse/2800-kongens%20l...,2800
2,Danmarksvej 43A,55.794678,12.517936,7695000,Villa,182,42280,D,https://www.dingeo.dk/adresse/2800-kongens%20l...,2800
3,Stengårdsvænge 103,55.759154,12.482144,5295000,Rækkehus,102,51911,C,https://www.dingeo.dk/adresse/2800-kongens%20l...,2800
4,Buddingevej 110,55.758719,12.494712,8750000,Villa,188,46542,D,https://www.dingeo.dk/adresse/2800-kongens%20l...,2800


# Step 2 - Data processing

In [15]:
data.dropna(inplace=True)

# Step 3 - Feauture Selection
Selecting the relevant features (variables) that we want to use for predicting the price. We exclude any columns that are not useful or not available during prediction.

In [16]:
features = ['X', 'Y', 'Size', 'Type', 'Energy class', 'City']
target = 'Price'

# Step 4  - Preprocess and Split the data
A dictionary called label_encoders is created to store the label encoders for categorical features. The code then iterates over each feature in features and checks if the feature's data type is 'object' (indicating a categorical feature). If it is, a LabelEncoder is created for that feature, and the feature values in the data DataFrame are transformed using label_encoders[feature].fit_transform(data[feature]).

Spliting the data into training and testing sets to evaluate the performance of your machine learning model. The training set will be used to train the model, and the testing set will be used to evaluate its performance.

In [17]:
# Preprocess the data
label_encoders = {}
for feature in features:
    if data[feature].dtype == 'object':
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

print(label_encoders)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

{'Type': LabelEncoder(), 'Energy class': LabelEncoder()}


# Step 5 - Train a model
Choose a suitable machine learning algorithm for your task, such as linear regression, decision tree, or random forest. Train the model on the training data.

In [18]:
# Train the random forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

# Saving the model
for later use

In [19]:
#joblib.dump(model, 'RFG_Model')

# Step 6 - Evaluate the model

In [20]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print(f"R2-score {r2}")

mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print(f"Root Mean Squared Error (RMSE): {rmse}")

R2-score 0.730780767260184
Root Mean Squared Error (RMSE): 3614408.2651437623


# Step 7 - Make predictions

In [21]:
# Apply label encoding to the new house data 
new_house = pd.DataFrame([[56.034845, 12.591295, 68, 'Ejerlejlighed', 'D', 3000]], columns=features)
for feature in features:
    if new_house[feature].dtype == 'object':
        new_house[feature] = label_encoders[feature].transform(new_house[feature])
        
# Example prediction for a new house
predicted_price = model.predict(new_house)
locale.setlocale(locale.LC_ALL, 'da_DK.UTF-8')
formatted_price = locale.currency(predicted_price[0], grouping=True)
print(f"Predicted price for the house: {formatted_price}")

Predicted price for the house: kr 1.888.400,00


## Other visualizations